In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression as OLS
import datetime,math
from random import *
from copy import *
seed(123) 

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
##############################################################
##############################################################

In [4]:
# Converts Datetime to Decimal Year: 
def dt2dec(dt_obj,base_year=2000,rounded=True):
    try:    
        Year = dt_obj.year
    except: 
        dt_obj = pd.to_datetime(dt_obj)
        Year = dt_obj.year
    year_beg = datetime.datetime(Year,1,1)
    year_end = datetime.datetime(Year+1,1,1)
    tot_sec  = (year_end-year_beg).total_seconds()
    elapsed  = (dt_obj  -year_beg).total_seconds()
    Portion  = elapsed/float(tot_sec) 
    dec_year = Year+Portion-base_year
    if rounded: dec_year = round(dec_year,7) # To Second.
    return dec_year

# Converts Decimal Year to DateTime Object 
def dec2dt(dec_year,base_year=2000):
    Portion = dec_year-int(dec_year)
    Year = int(dec_year)+base_year   
    year_beg = datetime.datetime(Year,1,1)
    year_end = datetime.datetime(Year+1,1,1)
    tot_sec  = (year_end-year_beg).total_seconds()
    elapsed  = int(round(tot_sec*Portion)) 
    delta = datetime.timedelta(seconds=elapsed)
    dt_obj = year_beg+delta
    return dt_obj    

In [5]:
# Converts a distribution to Z-Score Values: 
def ConvertToZ(thing):
    a = np.array(thing)
    return (a-a.mean())/a.std() 

# Subtracts a mean from a distribution:
def DeMean(thing):
    a = np.array(thing)
    return a-a.mean()  

def SeasonEffect(dec_yr,offset=0.0,cycles_per_yr=1.0):
    sea = math.cos((cycles_per_yr*2.0)*(dec_yr-offset)*math.pi)
    return round(sea,9) 

In [6]:
##############################################################
##############################################################

In [7]:
# LETS Make a data set!!

In [8]:
fn = 'DATA/ZipInfo.csv'
Zip = pd.read_csv(fn)

In [9]:
Zip['NW_VAL'] = Zip['LAT']-Zip['LON'] 
Zip['NW_Z']   = ConvertToZ(Zip['NW_VAL']) 

In [10]:
N_Students = 217483
N_Zipcodes = len(Zip) 

Sample_Ave = N_Students/float(N_Zipcodes) 
Zip['POP_RATIO'] = Zip['POP_TOT']/Zip['POP_TOT'].mean() 
Zip['LG_SAMPLE'] = [int(round(ratio*Sample_Ave*3.001)) for ratio in Zip['POP_RATIO']] 

BaseSample = []
for z,n in zip(Zip['ZIP'],Zip['LG_SAMPLE']): 
    for _ in range(n):
        BaseSample.append(z)
        
MidI = int(len(BaseSample)/2.0)
for _ in range(10):
    BaseSample = BaseSample[:MidI]+BaseSample[MidI:]
    shuffle(BaseSample) 
    
print 'BaseSample Len:',len(BaseSample) 

BaseSample Len: 652686


In [11]:
Zip.head(5)

,ZIP,LAT,LON,POP_TOT,NW_VAL,NW_Z,POP_RATIO,LG_SAMPLE
0,85607,31.480,-109.476,21131.0,140.956,0.745408,0.942349,130
1,29330,35.052,-81.808,7973.0,116.860,-0.803786,0.355561,49
2,98011,47.754,-122.201,26380.0,169.955,2.609828,1.176431,162
3,31757,30.870,-83.872,9771.0,114.742,-0.939958,0.435743,60
4,10901,41.161,-74.139,21760.0,115.300,-0.904083,0.970400,134


In [12]:
Center = [37.44,-92.40]  
lat,lon = Zip['LAT'].mean(),Zip['LON'].mean()
print round(lat,4),round(lon,4)

37.28 -92.082


In [13]:
Zip2 = deepcopy(Zip)
Zip2['RANDOM_G'] = [gauss(0,1)   for _ in range(len(Zip2))] 
Zip2['RANDOM_U'] = [uniform(0,1) for _ in range(len(Zip2))] 
Zip2.index = [int(a) for a in Zip2['ZIP']] 

In [14]:
Zip2.head()

,ZIP,LAT,LON,POP_TOT,NW_VAL,NW_Z,POP_RATIO,LG_SAMPLE,RANDOM_G,RANDOM_U
85607,85607,31.480,-109.476,21131.0,140.956,0.745408,0.942349,130,-0.989999,0.902590
29330,29330,35.052,-81.808,7973.0,116.860,-0.803786,0.355561,49,0.851161,0.386959
98011,98011,47.754,-122.201,26380.0,169.955,2.609828,1.176431,162,0.538664,0.015808
31757,31757,30.870,-83.872,9771.0,114.742,-0.939958,0.435743,60,0.318246,0.483532
10901,10901,41.161,-74.139,21760.0,115.300,-0.904083,0.970400,134,-0.305882,0.930786


In [15]:
zip2state,zip2nwz,zip2rge = {},{},{} 
zip2lat,zip2lon = {},{} 
for r in Zip2.iterrows():
    d = r[1]
    z = d['ZIP'] 
    zip2nwz[z]   = d['NW_Z']
    zip2rge[z]   = d['RANDOM_G'] 
    zip2lat[z]   = d['LAT']
    zip2lon[z]   = d['LON'] 

In [16]:
##############################################################
##############################################################

In [17]:
Alphas = set(list('ABCDEFGHJKLMNOPQRSTUVXYZ'))
LenNames = 5
NumNames = 500000
Names = [''.join(sample(Alphas,LenNames)) for _ in range(NumNames)] 
Names = list(set(Names))

MidI = int(len(Names)/2.0)
for _ in range(10):
    Names = Names[:MidI]+Names[MidI:]
    shuffle(Names)

print 'Random Names: ',NumNames 
print 'Number Unique:',len(Names)
print 'Ratio Unique: ',len(Names)/float(NumNames)

Random Names:  500000
Number Unique: 476191
Ratio Unique:  0.952382


In [18]:
##############################################################
##############################################################

In [19]:
# DATA GENERATION PARAMETERS
NumStu  = 237546 
MinDecD = 16.0
MaxDecD = 18.0 
Treatmt = 0.19  

In [20]:
# Create a Test Scores DataFrame: 
TS = pd.DataFrame()
TS['STU_ID'] = Names[:NumStu] 
TS['ZIP']    = BaseSample[:NumStu] 
IRange = range(len(TS)) 

#TS['STATE']  = [zip2state[z] for z in TS['ZIP']] # State Associated w/Zip 
TS['ZIP_RE'] = [zip2rge[z]   for z in TS['ZIP']] # ZipCode Random Effect
TS['NW_RE']  = [zip2nwz[z]   for z in TS['ZIP']] # NorthWest Random Effect
TS['LAT']    = [zip2lat[z]   for z in TS['ZIP']] 
TS['LON']    = [zip2lon[z]   for z in TS['ZIP']] 

TS['STU_ST'] = [round(max( 0.0,gauss( 20,20))) for _ in IRange]  # Student Study Time
TS['STU_IQ'] = [round(max(70.0,gauss(105,15))) for _ in IRange]  # Student IQ Score 
TS['STU_BF'] = [int(bool(uniform(0,1)<0.6))    for _ in IRange]  # Student Eats Breakfast 
TS['STU_SK'] = [randint(1,99)                  for _ in IRange]  # Student Skill at testing
TS['STU_LK'] = [randint(1,99)                  for _ in IRange]  # Student Luck on test day 

TS['DEC_DT'] = [round(uniform(MinDecD,MaxDecD),3) for _ in IRange] 
TS['DATE']   = [str(dec2dt(dec))[:10] for dec in TS['DEC_DT']] 
TS['LIN_TM'] = [dec-MinDecD for dec in TS['DEC_DT']]                # Linear Time
TS['SEA_TM'] = [SeasonEffect(dec,+0.234) for dec in TS['DEC_DT']]   # Season Time 

In [21]:
TS.head(5)  

,STU_ID,ZIP,ZIP_RE,NW_RE,LAT,LON,STU_ST,STU_IQ,STU_BF,STU_SK,STU_LK,DEC_DT,DATE,LIN_TM,SEA_TM
0,GQCPF,70006,-2.032357,-0.588792,30.013,-90.191,3.0,94.0,0,74,43,17.693,2017-09-10,1.693,-0.967001
1,NBXEV,71263,-0.057403,-0.325514,32.873,-91.426,13.0,87.0,0,20,17,16.241,2016-03-29,0.241,0.999033
2,EARNY,48223,-0.007884,-0.239362,42.393,-83.246,28.0,116.0,1,11,97,16.841,2016-11-03,0.841,-0.782391
3,LXPHO,80524,-0.721136,1.048933,40.648,-105.029,25.0,104.0,1,55,16,16.144,2016-02-22,0.144,0.844328
4,KGRBJ,70726,0.083929,-0.516077,30.433,-90.902,0.0,118.0,1,78,24,17.974,2017-12-22,1.974,-0.062791


In [22]:
##############################################################
##############################################################

In [23]:
# Generate the "Magic Cereal" Variable:
MinNW,MaxNW = TS['NW_RE'].min(),TS['NW_RE'].max() 
DifNW = MaxNW-MinNW  # Diference of NW maximums 
ZCP = np.array([(a-MinNW)/DifNW for a in TS['NW_RE']])  # Zipcode Propensity
MaxLT = TS['LIN_TM'].max()  # Max Linear Time 
LTP  = np.array([(t/MaxLT) for t in TS['LIN_TM']])      # Linear Time Propensity
BFP = TS['STU_BF']                                      # Breakfast Propensity
TRP = np.array([uniform(0.01,0.99) for _ in IRange])    # Treatment Random Propensity                 
MCP = ((TRP*.7)+(BFP*.3))*(ZCP**1.5)*(LTP**1.5)         # Magic Cereal Propensity   

In [24]:
TS['MCP'] = MCP 
thresh = np.percentile(TS['MCP'],100.0*(1.0-Treatmt)) 
TS['MCT'] = [int(bool(a>thresh)) for a in TS['MCP']]    # Magic Cereal Treatment  

In [25]:
print 'Treatment Portion:'
print round(TS['MCT'].mean(),10)

Treatment Portion:
0.1900010945


In [26]:
##############################################################
##############################################################

In [27]:
Notes = '''

Factors Affecting Test Performance:

# STUDENT EFFECTS:
TS['STU_ST'] # Student Study Time
TS['STU_IQ'] # Student IQ Score 
TS['STU_BF'] # Student Eats Breakfast 
TS['STU_SK'] # Student Skill at testing
TS['STU_LK'] # Student Luck on test day

# ZIP CODE EFFECTS: 
TS['ZIP_RE'] # ZipCode Random Effect
TS['NW_RE']  # NorthWest Random Effect 

# TIME EFFECTS:
TS['LIN_TM'] # Linear Time
TS['SEA_TM'] # Season Time  

# CEREAL EFFECTS:
TS['MCT'] # Magic Cereal Treatment 

''' 

In [28]:
# FINALLY, CALCULATE THE TEST PERFORMANCE (TP):

# Initialize all test scores to 100.0:
TS['SCORE'] = np.array([100.0 for _ in IRange])  

# Factor-In Student Variables:
TS['SCORE'] = TS['SCORE'] + (ConvertToZ(TS['STU_ST']) *  7.1)   
TS['SCORE'] = TS['SCORE'] + (ConvertToZ(TS['STU_IQ']) *  7.5)  
TS['SCORE'] = TS['SCORE'] + (ConvertToZ(TS['STU_SK']) *  4.1)  
TS['SCORE'] = TS['SCORE'] + (ConvertToZ(TS['STU_LK']) *  5.8)  
TS['SCORE'] = TS['SCORE'] + (    DeMean(TS['STU_BF']) *  6.9)  

# Factor-in Location Effects:
TS['SCORE'] = TS['SCORE'] + (ConvertToZ(TS['ZIP_RE']) *  6.2)
TS['SCORE'] = TS['SCORE'] + (ConvertToZ(TS['NW_RE' ]) * 10.7) 

# Factor-in Time Effects: 
TS['SCORE'] = TS['SCORE'] + (    DeMean(TS['LIN_TM']) * -4.8)  
TS['SCORE'] = TS['SCORE'] + (    DeMean(TS['SEA_TM']) *  3.2)  

# Factor-in the Magic Cereal Treatment Effect:   
TS['SCORE'] = TS['SCORE'] + (    DeMean(TS['MCT'])    * -3.14159)  
TS['SCORE'] = [round(a,1) for a in TS['SCORE']] 

In [29]:
# We can look at the final distribution of 'SCORE' values:  
TS['SCORE'].describe()

count    237546.000000
mean         99.999965
std          18.152821
min          32.300000
25%          87.300000
50%          99.300000
75%         112.100000
max         187.300000
Name: SCORE, dtype: float64

In [30]:
##############################################################
##############################################################

In [31]:
TS.head().T 

,0,1,2,3,4
STU_ID,GQCPF,NBXEV,EARNY,LXPHO,KGRBJ
ZIP,70006,71263,48223,80524,70726
ZIP_RE,-2.03236,-0.057403,-0.00788356,-0.721136,0.0839285
NW_RE,-0.588792,-0.325514,-0.239362,1.04893,-0.516077
LAT,30.013,32.873,42.393,40.648,30.433
LON,-90.191,-91.426,-83.246,-105.029,-90.902
STU_ST,3,13,28,25,0
STU_IQ,94,87,116,104,118
STU_BF,0,0,1,1,1
STU_SK,74,20,11,55,78


In [32]:
CleanCols = ['DATE','SCORE','STU_ID','MCT','STU_IQ','STU_ST','STU_BF','ZIP']    
TS2 = deepcopy(TS) 
TS2 = TS2[CleanCols] 

In [33]:
fn = 'DATA/TestData.csv'
TS2.to_csv(fn,index=False) 

In [34]:
##############################################################
##############################################################

In [35]:
# [END] 